# 整然データを作る

[前回の演習](../exercise_5/intro-to-pandas.ipynb)に続いて、この演習でも整然データの作成を練習します。

この部分は、[Pandas for Everyone (Chen, 2022)](https://github.com/chendaniely/pandas_for_everyone) を参考にしています。

整然データの4つの条件を再掲します。

1. 1つの列は、1つの変数を表す。
2. 1つの行は、1つの観測を表す。
3. 1つのセル（特定の列の特定の行）は、1つの値を表す。
4. 1つの表は、1つの観測単位を持つ（異なる観察単位が混ざっていない）。

以下では、この4つの条件が満たされていない様々な状況と、pandasを使ってどのように操作すると整然データに変換することができるのかを見ていきます。

In [2]:
import pandas as pd

## 列に（変数ではなく）値が入っているとき

### 1つの列を残す

Pew Research Centerによる「アメリカ合衆国における収入と宗教のデータ」を使います。

In [74]:
pew = pd.read_csv('./data/pew.csv')

In [65]:
pew.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


このようなデータは「横持ち」（wide）データとも呼ばれます。これを整然とした「縦持ち」（long）データに変換するには、データフレームを融解（`melt`）します。プログラミング言語によっては、unpivotやgatherと言ったりします。

pandasではDataFrameの`.melt()`メソッドを使います。`.melt()`は、次のようなパラメータをとります。

* `id_vars`：そのまま残す変数群を指定する。
* `value_vars`：融解したい列を指定する。デフォルトでは、`id_vars`で指定しなかった列がすべて融解される。
* `var_name`：`value_vars`を融解して作る新しい列の名前（変数名）を指定できる。デフォルトでは`variable`になる。
* `value_name`：上記の`var_name`で値を表現することになる新しい列の名前を指定できる。デフォルトは`value`。

In [66]:
pew_long = pd.melt(pew, id_vars='religion')

In [67]:
pew_long

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


In [68]:
pew_long = pd.melt(pew, id_vars='religion', var_name='income', value_name='count')
pew_long.head()

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


### 複数の列を残す

データセットによっては、1列に集める形で残りの列を融解できません。例えば、以下のビルボードチャートのデータです。

In [69]:
billboard = pd.read_csv('./data/billboard.csv')

In [70]:
billboard.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


この場合、`id_vars`に複数の変数を指定することで複数の列を残すことができます。

In [71]:
billboard_long = pd.melt(
    billboard,
    id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
    var_name='week',
    value_name='rating'
)

In [72]:
billboard_long.head()

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0


In [73]:
billboard_long.shape

(24092, 7)

## 複数の変数を含む列があるとき

データセットの列が複数の変数を表現している場合もあります。例として、エボラ熱のデータセットを見ます。

In [75]:
ebola = pd.read_csv('./data/country_timeseries.csv')

In [76]:
ebola.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [82]:
ebola.iloc[:5, [0, 1, 5, 13]]

,Date,Day,Cases_Nigeria,Deaths_Nigeria
0,1/5/2015,289,NaN,NaN
1,1/4/2015,288,NaN,NaN
2,1/3/2015,287,NaN,NaN
3,1/2/2015,286,NaN,NaN
4,12/31/2014,284,NaN,NaN


列名の`Cases_Nigeria`と`Deaths_Nigeria`は、それぞれ実際には2つの変数を含んでいます。個別の状態である`Cases`と`Deaths`、そして国名の`Nigeria`です。

このデータも横持ちになっているので、まずは`.melt()`メソッドで融解します。

In [83]:
ebola_long = pd.melt(ebola, id_vars=['Date', 'Day'])
ebola_long.head()

,Date,Day,variable,value
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0
2,1/3/2015,287,Cases_Guinea,2769.0
3,1/2/2015,286,Cases_Guinea,NaN
4,12/31/2014,284,Cases_Guinea,2730.0


結果を見ると、`variable`の列をアンダースコアの前後に分ければ変数が2つある問題は解決できそうです。アンダースコアの前半を`status`（状態）、後半を`country`（国）として新しい列を作ることができます。

In [ ]:
'Cases_Guinea'.split('_')

['Cases', 'Guinea']

### 列を分割して個別に追加する

In [89]:
# 変数の文字列を取得し、文字列メソッドにアクセスして列の値を分解する
variable_split = ebola_long['variable'].str.split('_')

In [88]:
variable_split.head()

0    [Cases, Guinea]
1    [Cases, Guinea]
2    [Cases, Guinea]
3    [Cases, Guinea]
4    [Cases, Guinea]
Name: variable, dtype: object

全体はpandasのSeries、その中身の分割した値はリストとして返されます。

In [ ]:
type(variable_split)

pandas.core.series.Series

In [22]:
type(variable_split[0])

list

In [23]:
variable_split[0][1]

'Guinea'

In [24]:
status_values = variable_split.str.get(0)
status_values.head()

0    Cases
1    Cases
2    Cases
3    Cases
4    Cases
Name: variable, dtype: object

In [25]:
country_values = variable_split.str.get(1)
country_values.head()

0    Guinea
1    Guinea
2    Guinea
3    Guinea
4    Guinea
Name: variable, dtype: object

`.str.get()`メソッドでアンダースコア前後の文字列のリストが得られたので、これを使って新しい列を作ります。

In [26]:
ebola_long['status'] = status_values
ebola_long['country'] = country_values

In [27]:
ebola_long.head()

,Date,Day,variable,value,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


### 分割と結合を一度に行う

In [28]:
variable_split = ebola_long['variable'].str.split('_', expand=True)

In [29]:
type(variable_split)

pandas.core.frame.DataFrame

In [30]:
variable_split.head()

,0,1
0,Cases,Guinea
1,Cases,Guinea
2,Cases,Guinea
3,Cases,Guinea
4,Cases,Guinea


In [31]:
variable_split.columns = ['status_expand', 'country_expand']

In [32]:
variable_split.head()

,status_expand,country_expand
0,Cases,Guinea
1,Cases,Guinea
2,Cases,Guinea
3,Cases,Guinea
4,Cases,Guinea


In [33]:
ebola_long = pd.concat([ebola_long, variable_split], axis=1)

In [34]:
ebola_long.head()

,Date,Day,variable,value,status,country,status_expand,country_expand
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea,Cases,Guinea


## 行と列の両方に変数があるとき

変数が行と列の両方にまたがっている場合があります。例として、気象データを見てみます。

In [91]:
weather = pd.read_csv('./data/weather.csv')

In [92]:
weather.shape

(22, 35)

In [93]:
weather.iloc[:5, :11]

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN


このデータには、月ごとに毎日（`d1`から`d31`まで）記録された気温について、最小値と最大値があります。

ここで、基本を1列として扱いたく、最小値と最大値をそれぞれ個別の列として扱いたい状況を想定します。この場合、前者は横持ちを縦持ちにすること、後者は縦持ちを横持ちにすることで対応できます。

In [94]:
weather_melt = pd.melt(
    weather,
    id_vars=['id', 'year', 'month', 'element'],
    var_name='day',
    value_name='temp'
)

In [41]:
weather_melt.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


`.pivot_talbe()`を使って、`element`に対してピボット展開（横持ちへの変換）をします。

In [97]:
weather_tidy = weather_melt.pivot_table(
    index=['id', 'year', 'month', 'day'],
    columns='element',
    values='temp'
)

In [101]:
weather_tidy.head()

element                 tmax  tmin
id      year month day            
MX17004 2010 1     d30  27.8  14.5
             2     d11  29.7  13.4
                   d2   27.3  14.4
                   d23  29.9  10.7
                   d3   24.1  14.4

`.reset_index()`を使うことで、列の階層を平坦化できます。

In [99]:
weather_tidy.reset_index().head()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4


ここまでの操作は、すべて繋げて実行することもできます（メソッドチェーン）。

In [102]:
weather_tidy = (weather_melt
                .pivot_table(
                    index=['id', 'year', 'month', 'day'],
                    columns='element',
                    values='temp')
                .reset_index()
)

In [103]:
weather_tidy.head()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4


以上が、整然データの基本的な作り方でした。`.melt()`メソッド、`.pivot_table()`メソッド、`.str`アクセサの組み合わせで様々な変換ができることが分かったと思います。

より発展的な例は、[ノーベル賞受賞者のデータ](./nobel-laureates.ipynb)で確認してください。

---

【演習】

`./data/tokyo_covid19_positive_cases_by_agegroup.csv`を読み込み、


1. 年代を意味する変数を作り、値を1つの列にまとめてください。
2. 年月日から年を分離し、1つの列にしてください。


データの入手元：[東京都 新型コロナウイルス感染症年代別新規陽性者数](https://catalog.data.metro.tokyo.lg.jp/dataset/t000055d0000000393)

In [105]:
# your code goes here


---

この演習は以上です。